In [ ]:
!pip install catboost

In [2]:
import pandas as pd
import numpy as np
from tqdm import tqdm
import copy

In [3]:
train_ = pd.read_csv('Train.csv')
test_ = pd.read_csv('Test.csv')
submission_ = pd.read_csv('SampleSubmission.csv')

In [4]:
from sklearn.model_selection import KFold
fold_train = []
kf = KFold(n_splits=5, shuffle=False)
for r, (train_index, test_index) in enumerate(kf.split(train_)):
  test = train_.iloc[test_index]

  X_test = []
  X_test_columns = test.columns
  for v in test.values:
    info = v[:8]
    binary = v[8:]
    index = [k for k, i in enumerate(binary) if i == 1]
    for i in index:
      for k in range(len(binary)):
        if k == i:
          binary_transformed = list(copy.copy(binary))
          binary_transformed[i] = 0
          X_test.append(list(info) + binary_transformed + [8+k])

  X_test = pd.DataFrame(X_test)
  X_test.columns = ['ID', 'join_date', 'sex', 'marital_status', 'birth_year', 'branch_code',
          'occupation_code', 'occupation_category_code', 'P5DA', 'RIBP', '8NN1',
          '7POT', '66FJ', 'GYSR', 'SOP4', 'RVSZ', 'PYUQ', 'LJR9', 'N2MW', 'AHXO',
          'BSTQ', 'FM3X', 'K6QO', 'QBOL', 'JWFN', 'JZ9D', 'J9JW', 'GHYX', 'ECY3', 'product_num']

  X_test['ID'] = [str(r)+'_'+str(i) for i in range(X_test.shape[0])]

  test = []
  test_columns = X_test.columns

  for v in X_test.values:
    info = v[:8]
    binary = v[8:-1]
    num = v[-1]
    index = [k for k, i in enumerate(binary) if i == 1]
    for k in range(len(binary)):
        if k not in index:
          if 8+k == num:
            test.append(list(info) + list(binary) + [test_columns[8+k]] + [1])
          else:
            test.append(list(info) + list(binary) + [test_columns[8+k]] + [0])
  test = pd.DataFrame(test)
  test.columns = ['ID', 'join_date', 'sex', 'marital_status', 'birth_year', 'branch_code',
        'occupation_code', 'occupation_category_code', 'P5DA', 'RIBP', '8NN1',
        '7POT', '66FJ', 'GYSR', 'SOP4', 'RVSZ', 'PYUQ', 'LJR9', 'N2MW', 'AHXO',
        'BSTQ', 'FM3X', 'K6QO', 'QBOL', 'JWFN', 'JZ9D', 'J9JW', 'GHYX', 'ECY3', 'product_pred', 'target']

  test = test[['ID', 'product_pred', 'target']]
  test['ID X PCODE'] = test['ID'] + ' X ' + test['product_pred']
  del test['ID'], test['product_pred']
  test.columns = ['Label', 'ID X PCODE']

  submit1 = pd.read_csv('0_fold' + str(r) + '.csv')
  submit2 = pd.read_csv('1_fold' + str(r) + '.csv')
  submit3 = pd.read_csv('2_fold' + str(r) + '.csv')

  test = test.merge(submit1, how='left', on='ID X PCODE')
  test = test.merge(submit2, how='left', on='ID X PCODE')
  test = test.merge(submit3, how='left', on='ID X PCODE')

  test.columns = ['target', 'ID X PCODE', 'Label1', 'Label2', 'Label3']

  fold_train.append(test)

fold_train = pd.concat(fold_train)

In [5]:
test = []
test_columns = test_.columns

for v in test_.values:
  info = v[:8]
  binary = v[8:]
  index = [k for k, i in enumerate(binary) if i == 1]
  for k in range(len(binary)):
    if k not in index:
      test.append(list(info) + list(binary) + [test_columns[8+k]])
test = pd.DataFrame(test)
test.columns = ['ID', 'join_date', 'sex', 'marital_status', 'birth_year', 'branch_code',
        'occupation_code', 'occupation_category_code', 'P5DA', 'RIBP', '8NN1',
        '7POT', '66FJ', 'GYSR', 'SOP4', 'RVSZ', 'PYUQ', 'LJR9', 'N2MW', 'AHXO',
        'BSTQ', 'FM3X', 'K6QO', 'QBOL', 'JWFN', 'JZ9D', 'J9JW', 'GHYX', 'ECY3', 'product_pred',]

test = test[['ID', 'product_pred']]
test['ID X PCODE'] = test['ID'] + ' X ' + test['product_pred']
del test['ID'], test['product_pred']

submit1 = pd.read_csv('0_main.csv')
submit2 = pd.read_csv('1_main.csv')
submit3 = pd.read_csv('2_main.csv')

test = test.merge(submit1, how='left', on='ID X PCODE')
test = test.merge(submit2, how='left', on='ID X PCODE')
test = test.merge(submit3, how='left', on='ID X PCODE')

test.columns = ['ID X PCODE', 'Label1', 'Label2', 'Label3']

In [12]:
fold_train.head()

,target,Label1,Label2,Label3,ID,PCODE
0,0,5.157589e-07,1.372533e-07,0.005326,0_0,P5DA
1,0,4.155255e-05,1.667119e-07,0.003657,0_0,RIBP
2,0,3.115225e-07,1.372533e-07,0.005326,0_0,8NN1
3,0,2.373249e-05,1.667119e-07,0.005326,0_0,7POT
4,0,4.203432e-06,1.667119e-07,0.005326,0_0,66FJ


In [13]:
test.head()

,Label1,Label2,Label3,ID,PCODE
0,0.000003,0.000915,0.005931,F86J5PC,P5DA
1,0.000655,0.000955,0.003241,F86J5PC,RIBP
2,0.000007,0.000915,0.005931,F86J5PC,8NN1
3,0.000062,0.000915,0.005931,F86J5PC,7POT
4,0.000229,0.000915,0.005931,F86J5PC,66FJ


new

In [11]:
fold_train['ID'] = fold_train['ID X PCODE'].apply(lambda x: x.split(' X ')[0])
fold_train['PCODE'] = fold_train['ID X PCODE'].apply(lambda x: x.split(' X ')[1])

test['ID'] = test['ID X PCODE'].apply(lambda x: x.split(' X ')[0])
test['PCODE'] = test['ID X PCODE'].apply(lambda x: x.split(' X ')[1])

del fold_train['ID X PCODE'], test['ID X PCODE']

In [21]:
new_df = pd.DataFrame(np.zeros((fold_train['ID'].nunique(), fold_train['PCODE'].nunique()*3+1)))
new_df.index = fold_train['ID'].unique()
new_df.columns = list(fold_train['PCODE'].unique() + '_0') + list(fold_train['PCODE'].unique() + '_1') + list(fold_train['PCODE'].unique() + '_2') + ['target']

In [22]:
for v in tqdm(fold_train.values):
   new_df.loc[v[4], v[5]+'_0'] = v[1]
   new_df.loc[v[4], v[5]+'_1'] = v[2]
   new_df.loc[v[4], v[5]+'_2'] = v[3]
   if v[0] == 1:
     new_df.loc[v[4], 'target'] = v[5]

100%|██████████| 1298055/1298055 [51:36<00:00, 419.25it/s]


In [23]:
new_df

,P5DA_0,RIBP_0,8NN1_0,7POT_0,66FJ_0,GYSR_0,SOP4_0,RVSZ_0,PYUQ_0,LJR9_0,N2MW_0,AHXO_0,BSTQ_0,FM3X_0,QBOL_0,JWFN_0,JZ9D_0,J9JW_0,GHYX_0,ECY3_0,K6QO_0,P5DA_1,RIBP_1,8NN1_1,7POT_1,66FJ_1,GYSR_1,SOP4_1,RVSZ_1,PYUQ_1,LJR9_1,N2MW_1,AHXO_1,BSTQ_1,FM3X_1,QBOL_1,JWFN_1,JZ9D_1,J9JW_1,GHYX_1,ECY3_1,K6QO_1,P5DA_2,RIBP_2,8NN1_2,7POT_2,66FJ_2,GYSR_2,SOP4_2,RVSZ_2,PYUQ_2,LJR9_2,N2MW_2,AHXO_2,BSTQ_2,FM3X_2,QBOL_2,JWFN_2,JZ9D_2,J9JW_2,GHYX_2,ECY3_2,K6QO_2,target,target
0_0,5.157589e-07,0.000042,3.115225e-07,0.000024,0.000004,5.216248e-08,0.000003,0.999230,0.000063,0.000010,0.000084,0.000056,0.000043,1.886010e-06,0.000065,0.000008,0.000019,0.000006,0.000309,0.000021,0.000000,1.372533e-07,1.667119e-07,1.372533e-07,1.667119e-07,1.667119e-07,1.372533e-07,1.667119e-07,0.999993,1.667119e-07,1.667119e-07,1.667119e-07,1.667119e-07,1.667119e-07,1.372533e-07,0.000003,1.667119e-07,1.667119e-07,1.667119e-07,1.667119e-07,1.667119e-07,0.000000,0.005326,0.003657,0.005326,0.005326,0.005326,0.005326,0.005326,0.970451,0.007609,0.005326,0.013050,0.005326,0.005326,0.005326,0.050242,0.005326,0.008679,0.008679,0.005326,0.005419,0.000000,0.0,RVSZ
0_1,3.730573e-06,0.000182,1.310676e-06,0.000045,0.000009,1.253211e-07,0.000012,0.000000,0.000488,0.000030,0.000234,0.000122,0.000107,6.568602e-06,0.002347,0.000103,0.000050,0.000021,0.003126,0.000397,0.992711,6.544655e-04,7.948214e-04,6.544655e-04,7.948214e-04,7.948214e-04,6.544655e-04,7.948214e-04,0.000000,1.076238e-03,7.948214e-04,7.948214e-04,7.948214e-04,7.948214e-04,6.544655e-04,0.005845,7.948214e-04,7.948214e-04,7.948214e-04,7.948214e-04,7.948214e-04,0.996997,0.005326,0.003657,0.005326,0.005326,0.005326,0.005326,0.005326,0.000000,0.007609,0.005326,0.013050,0.005326,0.005326,0.005326,0.050242,0.005326,0.008679,0.008679,0.005326,0.005419,0.969610,0.0,K6QO
0_2,1.736840e-06,0.000036,9.705029e-07,0.000009,0.000015,7.107538e-08,0.000020,0.998565,0.000167,0.000031,0.000120,0.000044,0.000014,1.829235e-06,0.000258,0.000004,0.000040,0.000022,0.000536,0.000106,0.000000,1.244332e-07,1.244332e-07,1.244332e-07,1.244332e-07,1.244332e-07,1.244332e-07,1.244332e-07,0.999990,1.244332e-07,1.244332e-07,1.244332e-07,1.244332e-07,1.244332e-07,1.244332e-07,0.000008,1.244332e-07,8.276140e-08,8.276140e-08,1.244332e-07,1.244332e-07,0.000000,0.006220,0.004232,0.006220,0.006220,0.006220,0.006220,0.006220,0.947804,0.008957,0.006220,0.009398,0.006220,0.006220,0.006220,0.105656,0.006220,0.040512,0.040512,0.006220,0.007510,0.000000,0.0,RVSZ
0_3,1.642618e-05,0.000232,6.846940e-06,0.000020,0.000043,2.126823e-07,0.000086,0.000000,0.001488,0.000098,0.000416,0.000182,0.000041,5.862713e-06,0.013347,0.000058,0.000126,0.000100,0.006227,0.001858,0.975645,7.970565e-04,7.970565e-04,7.970565e-04,7.970565e-04,7.970565e-04,7.970565e-04,7.970565e-04,0.000000,1.062328e-03,7.970565e-04,7.970565e-04,7.970565e-04,7.970565e-04,7.970565e-04,0.017867,7.970565e-04,5.443425e-04,5.443425e-04,7.970565e-04,7.970565e-04,0.989544,0.006220,0.004232,0.006220,0.006220,0.006220,0.006220,0.006220,0.000000,0.008957,0.006220,0.009398,0.006220,0.006220,0.006220,0.105656,0.006220,0.040512,0.040512,0.006220,0.007510,0.931375,0.0,K6QO
0_4,3.268592e-05,0.040642,6.309915e-03,0.000100,0.054204,5.485520e-06,0.841872,0.000000,0.000399,0.000525,0.000127,0.000462,0.000004,4.467523e-06,0.000607,0.000010,0.000031,0.000023,0.001578,0.000000,0.052317,2.990665e-03,7.704699e-02,2.990665e-03,9.139571e-03,2.529048e-01,2.990665e-03,6.091714e-01,0.000000,2.071340e-02,8.514295e-03,3.630219e-03,3.630219e-03,3.630219e-03,2.990665e-03,0.003885,3.630219e-03,3.630219e-03,3.630219e-03,3.630219e-03,0.000000e+00,0.260917,0.005918,0.117825,0.010656,0.014407,0.240564,0.005918,0.628918,0.000000,0.025040,0.015523,0.005918,0.005918,0.005918,0.005918,0.034525,0.005918,0.005918,0.005918,0.005918,0.000000,0.210887,0.0,SOP4
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,

In [26]:
new_df_test = pd.DataFrame(np.zeros((test['ID'].nunique(), test['PCODE'].nunique()*3)))
new_df_test.index = test['ID'].unique()
new_df_test.columns = list(test['PCODE'].unique() + '_0') + list(test['PCODE'].unique() + '_1') + list(test['PCODE'].unique() + '_2') 

In [27]:
test

,Label1,Label2,Label3,ID,PCODE
0,0.000003,9.150753e-04,0.005931,F86J5PC,P5DA
1,0.000655,9.546101e-04,0.003241,F86J5PC,RIBP
2,0.000007,9.150753e-04,0.005931,F86J5PC,8NN1
3,0.000062,9.150753e-04,0.005931,F86J5PC,7POT
4,0.000229,9.150753e-04,0.005931,F86J5PC,66FJ
...,...,...,...,...,...
197142,0.000027,1.398092e-07,0.006057,WHII93K,JWFN
197143,0.000065,1.791148e-07,0.047383,WHII93K,JZ9D
197144,0.000064,1.791148e-07,0.047383,WHII93K,J9JW
197145,0.000725,1.875537e-07,0.006337,WHII93K,GHYX


In [28]:
for v in tqdm(test.values):
   new_df_test.loc[v[3], v[4]+'_0'] = v[0]
   new_df_test.loc[v[3], v[4]+'_1'] = v[1]
   new_df_test.loc[v[3], v[4]+'_2'] = v[2]

100%|██████████| 197147/197147 [02:00<00:00, 1640.56it/s]


In [29]:
new_df_test

,P5DA_0,RIBP_0,8NN1_0,7POT_0,66FJ_0,GYSR_0,SOP4_0,PYUQ_0,LJR9_0,N2MW_0,AHXO_0,BSTQ_0,FM3X_0,K6QO_0,QBOL_0,JWFN_0,JZ9D_0,J9JW_0,GHYX_0,ECY3_0,RVSZ_0,P5DA_1,RIBP_1,8NN1_1,7POT_1,66FJ_1,GYSR_1,SOP4_1,PYUQ_1,LJR9_1,N2MW_1,AHXO_1,BSTQ_1,FM3X_1,K6QO_1,QBOL_1,JWFN_1,JZ9D_1,J9JW_1,GHYX_1,ECY3_1,RVSZ_1,P5DA_2,RIBP_2,8NN1_2,7POT_2,66FJ_2,GYSR_2,SOP4_2,PYUQ_2,LJR9_2,N2MW_2,AHXO_2,BSTQ_2,FM3X_2,K6QO_2,QBOL_2,JWFN_2,JZ9D_2,J9JW_2,GHYX_2,ECY3_2,RVSZ_2
F86J5PC,3.448381e-06,0.000655,6.706585e-06,0.000062,0.000229,8.432026e-07,0.000248,0.000905,0.000077,0.000157,0.000047,0.000090,1.398135e-05,0.974558,0.012990,2.427463e-04,0.000114,0.000098,0.004727,0.004768,0.000000,9.150753e-04,9.546101e-04,9.150753e-04,9.150753e-04,9.150753e-04,9.150753e-04,9.150753e-04,9.535783e-04,9.150753e-04,9.150753e-04,9.150753e-04,9.150753e-04,9.150753e-04,9.947655e-01,0.015516,9.150753e-04,9.195758e-04,9.195758e-04,9.546101e-04,2.362449e-03,0.000000,0.005931,0.003241,0.005931,0.005931,0.005931,0.005931,0.005931,0.017125,0.005931,0.005931,0.005931,0.005931,0.005931,0.940500,0.075687,0.005931,0.062743,0.062743,0.021175,0.063528,0.000000
H6141K3,2.255677e-05,0.002551,6.020528e-05,0.000261,0.000835,5.955773e-06,0.000858,0.004132,0.000633,0.003947,0.001218,0.000947,5.590510e-04,0.000000,0.457742,1.200430e-01,0.000879,0.000435,0.231642,0.169062,0.000000,2.389965e-03,9.158084e-03,5.507729e-03,5.507729e-03,5.507729e-03,2.389965e-03,5.507729e-03,2.260222e-02,5.507729e-03,5.507729e-03,5.507729e-03,5.507729e-03,5.507729e-03,0.000000e+00,0.335141,6.217645e-03,6.436430e-03,6.436430e-03,6.670744e-02,1.246313e-01,0.000000,0.001977,0.000997,0.001977,0.001977,0.001977,0.001977,0.001977,0.002710,0.001977,0.001977,0.001977,0.001977,0.001977,0.000000,0.047427,0.001977,0.003016,0.003016,0.011105,0.002136,0.000000
RBAYUXZ,9.016221e-06,0.001489,1.846823e-04,0.000240,0.000482,6.897419e-07,0.000983,0.005514,0.001675,0.005655,0.003506,0.000076,1.153712e-05,0.000000,0.864153,1.042488e-02,0.000771,0.000248,0.012375,0.091810,0.000000,8.765891e-04,1.953284e-03,2.024125e-03,2.024125e-03,2.024125e-03,8.765891e-04,2.024125e-03,5.832326e-03,2.024125e-03,2.994801e-03,2.994801e-03,2.024125e-03,2.024125e-03,0.000000e+00,0.894262,2.173057e-03,2.005889e-03,2.005889e-03,4.360997e-03,6.473086e-02,0.000000,0.010302,0.010270,0.010302,0.010302,0.010302,0.010302,0.010302,0.012284,0.010302,0.036501,0.019680,0.010302,0.010302,0.000000,0.493766,0.010302,0.082461,0.082461,0.014499,0.057782,0.000000
KCBILBQ,3.044634e-06,0.000196,2.407980e-06,0.000031,0.000053,1.424339e-07,0.000186,0.000839,0.000165,0.000442,0.000160,0.000048,1.152660e-05,0.987249,0.005147,9.286298e-05,0.000071,0.000107,0.002911,0.002280,0.000000,7.771381e-04,7.771381e-04,7.771381e-04,7.771381e-04,7.771381e-04,7.771381e-04,7.771381e-04,8.161596e-04,7.771381e-04,7.771381e-04,7.771381e-04,7.771381e-04,7.771381e-04,9.872247e-01,0.008586,7.771381e-04,7.140519e-04,7.140519e-04,7.771381e-04,7.357029e-04,0.000000,0.004473,0.003212,0.004473,0.004473,0.004473,0.004473,0.004473,0.010779,0.004473,0.006082,0.004433,0.004473,0.004473,0.938789,0.055292,0.004473,0.065827,0.065827,0.004473,0.006185,0.000000
LSEC1ZJ,6.652264e-07,0.000015,2.327914e-06,0.000015,0.000009,2.653148e-08,0.000007,0.000956,0.000037,0.000050,0.000041,0.000073,1.224894e-06,0.000002,0.002332,4.234056e-07,0.000000,0.994268,0.000337,0.000058,0.001354,2.966270e-04,3.576807e-04,2.966270e-04,2.966270e-04,2.966270e-04,2.966270e-04,2.966270e-04,3.796224e-03,2.966270e-04,1.740154e-04,2.074780e-04,2.966270e-04,2.966270e-04,5.468323e-07,0.005859,2.966270e-04,0.000000e+00,9.900005e-01,2.966270e-04,4.654156e-04,0.000716,0.005723,0.005613,0.005723,0.005723,0.005723,0.005723,0.005723,0.094964,0.005723,0.008996,0.007764,0.005723,0.005723,0.181942,0.355406,0.005723,0.000000,0.801147,0.005723,0.026690,0.319137
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...

In [31]:
new_df

,P5DA_0,RIBP_0,8NN1_0,7POT_0,66FJ_0,GYSR_0,SOP4_0,RVSZ_0,PYUQ_0,LJR9_0,N2MW_0,AHXO_0,BSTQ_0,FM3X_0,QBOL_0,JWFN_0,JZ9D_0,J9JW_0,GHYX_0,ECY3_0,K6QO_0,P5DA_1,RIBP_1,8NN1_1,7POT_1,66FJ_1,GYSR_1,SOP4_1,RVSZ_1,PYUQ_1,LJR9_1,N2MW_1,AHXO_1,BSTQ_1,FM3X_1,QBOL_1,JWFN_1,JZ9D_1,J9JW_1,GHYX_1,ECY3_1,K6QO_1,P5DA_2,RIBP_2,8NN1_2,7POT_2,66FJ_2,GYSR_2,SOP4_2,RVSZ_2,PYUQ_2,LJR9_2,N2MW_2,AHXO_2,BSTQ_2,FM3X_2,QBOL_2,JWFN_2,JZ9D_2,J9JW_2,GHYX_2,ECY3_2,K6QO_2,target
0_0,5.157589e-07,0.000042,3.115225e-07,0.000024,0.000004,5.216248e-08,0.000003,0.999230,0.000063,0.000010,0.000084,0.000056,0.000043,1.886010e-06,0.000065,0.000008,0.000019,0.000006,0.000309,0.000021,0.000000,1.372533e-07,1.667119e-07,1.372533e-07,1.667119e-07,1.667119e-07,1.372533e-07,1.667119e-07,0.999993,1.667119e-07,1.667119e-07,1.667119e-07,1.667119e-07,1.667119e-07,1.372533e-07,0.000003,1.667119e-07,1.667119e-07,1.667119e-07,1.667119e-07,1.667119e-07,0.000000,0.005326,0.003657,0.005326,0.005326,0.005326,0.005326,0.005326,0.970451,0.007609,0.005326,0.013050,0.005326,0.005326,0.005326,0.050242,0.005326,0.008679,0.008679,0.005326,0.005419,0.000000,RVSZ
0_1,3.730573e-06,0.000182,1.310676e-06,0.000045,0.000009,1.253211e-07,0.000012,0.000000,0.000488,0.000030,0.000234,0.000122,0.000107,6.568602e-06,0.002347,0.000103,0.000050,0.000021,0.003126,0.000397,0.992711,6.544655e-04,7.948214e-04,6.544655e-04,7.948214e-04,7.948214e-04,6.544655e-04,7.948214e-04,0.000000,1.076238e-03,7.948214e-04,7.948214e-04,7.948214e-04,7.948214e-04,6.544655e-04,0.005845,7.948214e-04,7.948214e-04,7.948214e-04,7.948214e-04,7.948214e-04,0.996997,0.005326,0.003657,0.005326,0.005326,0.005326,0.005326,0.005326,0.000000,0.007609,0.005326,0.013050,0.005326,0.005326,0.005326,0.050242,0.005326,0.008679,0.008679,0.005326,0.005419,0.969610,K6QO
0_2,1.736840e-06,0.000036,9.705029e-07,0.000009,0.000015,7.107538e-08,0.000020,0.998565,0.000167,0.000031,0.000120,0.000044,0.000014,1.829235e-06,0.000258,0.000004,0.000040,0.000022,0.000536,0.000106,0.000000,1.244332e-07,1.244332e-07,1.244332e-07,1.244332e-07,1.244332e-07,1.244332e-07,1.244332e-07,0.999990,1.244332e-07,1.244332e-07,1.244332e-07,1.244332e-07,1.244332e-07,1.244332e-07,0.000008,1.244332e-07,8.276140e-08,8.276140e-08,1.244332e-07,1.244332e-07,0.000000,0.006220,0.004232,0.006220,0.006220,0.006220,0.006220,0.006220,0.947804,0.008957,0.006220,0.009398,0.006220,0.006220,0.006220,0.105656,0.006220,0.040512,0.040512,0.006220,0.007510,0.000000,RVSZ
0_3,1.642618e-05,0.000232,6.846940e-06,0.000020,0.000043,2.126823e-07,0.000086,0.000000,0.001488,0.000098,0.000416,0.000182,0.000041,5.862713e-06,0.013347,0.000058,0.000126,0.000100,0.006227,0.001858,0.975645,7.970565e-04,7.970565e-04,7.970565e-04,7.970565e-04,7.970565e-04,7.970565e-04,7.970565e-04,0.000000,1.062328e-03,7.970565e-04,7.970565e-04,7.970565e-04,7.970565e-04,7.970565e-04,0.017867,7.970565e-04,5.443425e-04,5.443425e-04,7.970565e-04,7.970565e-04,0.989544,0.006220,0.004232,0.006220,0.006220,0.006220,0.006220,0.006220,0.000000,0.008957,0.006220,0.009398,0.006220,0.006220,0.006220,0.105656,0.006220,0.040512,0.040512,0.006220,0.007510,0.931375,K6QO
0_4,3.268592e-05,0.040642,6.309915e-03,0.000100,0.054204,5.485520e-06,0.841872,0.000000,0.000399,0.000525,0.000127,0.000462,0.000004,4.467523e-06,0.000607,0.000010,0.000031,0.000023,0.001578,0.000000,0.052317,2.990665e-03,7.704699e-02,2.990665e-03,9.139571e-03,2.529048e-01,2.990665e-03,6.091714e-01,0.000000,2.071340e-02,8.514295e-03,3.630219e-03,3.630219e-03,3.630219e-03,2.990665e-03,0.003885,3.630219e-03,3.630219e-03,3.630219e-03,3.630219e-03,0.000000e+00,0.260917,0.005918,0.117825,0.010656,0.014407,0.240564,0.005918,0.628918,0.000000,0.025040,0.015523,0.005918,0.005918,0.005918,0.005918,0.034525,0.005918,0.005918,0.005918,0.005918,0.000000,0.210887,SOP4
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...

In [32]:
from sklearn.preprocessing import LabelEncoder
le = LabelEncoder()
le.fit(new_df['target'])
le.transform(new_df['target'])

array([19, 12, 19, ..., 12, 19, 12])

In [33]:
new_df['target'] = le.transform(new_df['target'])

In [149]:
from catboost import CatBoostClassifier

models = []
models.append(CatBoostClassifier(random_state=17, max_depth = 3))
models.append(CatBoostClassifier(random_state=18, num_trees=700, max_depth=5))
models.append(CatBoostClassifier(random_state=19, num_trees=500, max_depth = 3))
models.append(CatBoostClassifier(random_state=20, num_trees=1200, max_depth=3))
#models.append(CatBoostClassifier(random_state=21))

In [150]:
for i in range(len(models)):
  models[i].fit(new_df.drop(columns=['target',]), new_df['target'])

Learning rate set to 0.098003
0:	learn: 1.7934580	total: 395ms	remaining: 6m 34s
1:	learn: 1.5689456	total: 782ms	remaining: 6m 30s
2:	learn: 1.4161880	total: 1.2s	remaining: 6m 38s
3:	learn: 1.3014622	total: 1.6s	remaining: 6m 37s
4:	learn: 1.2090915	total: 2s	remaining: 6m 37s
5:	learn: 1.1357342	total: 2.38s	remaining: 6m 33s
6:	learn: 1.0718564	total: 2.77s	remaining: 6m 32s
7:	learn: 1.0200951	total: 3.15s	remaining: 6m 30s
8:	learn: 0.9728272	total: 3.5s	remaining: 6m 25s
9:	learn: 0.9332347	total: 3.88s	remaining: 6m 23s
10:	learn: 0.8927579	total: 4.26s	remaining: 6m 22s
11:	learn: 0.8610698	total: 4.62s	remaining: 6m 20s
12:	learn: 0.8327237	total: 4.95s	remaining: 6m 16s
13:	learn: 0.7999709	total: 5.28s	remaining: 6m 12s
14:	learn: 0.7805893	total: 5.66s	remaining: 6m 11s
15:	learn: 0.7604369	total: 6.01s	remaining: 6m 9s
16:	learn: 0.7377308	total: 6.34s	remaining: 6m 6s
17:	learn: 0.7215842	total: 6.67s	remaining: 6m 3s
18:	learn: 0.7025380	total: 7.02s	remaining: 6m 2s
19

In [151]:
df_answ = []
for num_pred in range(len(models)):
  proba = models[num_pred].predict_proba(new_df_test)
  y_test = pd.DataFrame(proba)
  y_test.columns = le.inverse_transform(y_test.columns)

  true_values = []
  for v in test_.values:
    binary = v[8:]
    index = [k for k, i in enumerate(binary) if i == 1]
    for k in test_.columns[8:][index]:
      true_values.append(v[0] + ' X ' + k)

  answer_mass = []
  for i in range(new_df_test.index.shape[0]):
    id = new_df_test.index[i]
    for c in y_test.columns:
      answer_mass.append([id + ' X ' + c, y_test[c].iloc[i]])

  df_answer = pd.DataFrame(answer_mass)
  df_answer.columns = ['ID X PCODE', 'Label']
  for i in range(df_answer.shape[0]):
    if df_answer['ID X PCODE'].iloc[i] in true_values:
      df_answer['Label'].iloc[i] = 1.0

  df_answ.append(df_answer)

/usr/local/lib/python3.6/dist-packages/pandas/core/indexing.py:671: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self._setitem_with_indexer(indexer, value)


In [152]:
df_answ

[            ID X PCODE     Label
 0       F86J5PC X 66FJ  0.000090
 1       F86J5PC X 7POT  0.000055
 2       F86J5PC X 8NN1  0.000006
 3       F86J5PC X AHXO  0.000113
 4       F86J5PC X BSTQ  0.000073
 ...                ...       ...
 209995  WHII93K X PYUQ  0.000081
 209996  WHII93K X QBOL  0.000297
 209997  WHII93K X RIBP  0.000040
 209998  WHII93K X RVSZ  0.999142
 209999  WHII93K X SOP4  0.000020
 
 [210000 rows x 2 columns],             ID X PCODE         Label
 0       F86J5PC X 66FJ  6.355554e-05
 1       F86J5PC X 7POT  2.744038e-05
 2       F86J5PC X 8NN1  7.238452e-07
 3       F86J5PC X AHXO  2.940785e-05
 4       F86J5PC X BSTQ  1.227218e-05
 ...                ...           ...
 209995  WHII93K X PYUQ  3.159026e-05
 209996  WHII93K X QBOL  1.123344e-04
 209997  WHII93K X RIBP  8.983423e-06
 209998  WHII93K X RVSZ  9.997764e-01
 209999  WHII93K X SOP4  2.405859e-06
 
 [210000 rows x 2 columns],             ID X PCODE     Label
 0       F86J5PC X 66FJ  0.000104
 1       F

In [153]:
df_answ_labels = []
for v in df_answ:
  df_answ_labels.append(v['Label'])

In [154]:
df_answ_labels

[0         0.000090
 1         0.000055
 2         0.000006
 3         0.000113
 4         0.000073
             ...   
 209995    0.000081
 209996    0.000297
 209997    0.000040
 209998    0.999142
 209999    0.000020
 Name: Label, Length: 210000, dtype: float64, 0         6.355554e-05
 1         2.744038e-05
 2         7.238452e-07
 3         2.940785e-05
 4         1.227218e-05
               ...     
 209995    3.159026e-05
 209996    1.123344e-04
 209997    8.983423e-06
 209998    9.997764e-01
 209999    2.405859e-06
 Name: Label, Length: 210000, dtype: float64, 0         0.000104
 1         0.000052
 2         0.000010
 3         0.000134
 4         0.000078
             ...   
 209995    0.000235
 209996    0.000382
 209997    0.000100
 209998    0.998599
 209999    0.000019
 Name: Label, Length: 210000, dtype: float64, 0         0.000086
 1         0.000050
 2         0.000008
 3         0.000104
 4         0.000051
             ...   
 209995    0.000084
 209996    0.000193
 

In [155]:
np.mean(df_answ_labels, axis=0)

array([8.58237805e-05, 4.60167210e-05, 6.05790303e-06, ...,
       4.35705445e-05, 9.99233151e-01, 1.43545491e-05])

In [156]:
df_answer = df_answ[0]
df_answer['Label'] = np.mean(df_answ_labels, axis=0)

In [157]:
df_answer.reset_index(drop=True, inplace=True)
df_answer.to_csv('submission_stack2.csv', index=False)

In [158]:
df_answer

,ID X PCODE,Label
0,F86J5PC X 66FJ,0.000086
1,F86J5PC X 7POT,0.000046
2,F86J5PC X 8NN1,0.000006
3,F86J5PC X AHXO,0.000095
4,F86J5PC X BSTQ,0.000053
...,...,...
209995,WHII93K X PYUQ,0.000108
209996,WHII93K X QBOL,0.000246
209997,WHII93K X RIBP,0.000044
209998,WHII93K X RVSZ,0.999233


OK